# Echo Transfer File Processor for ZIKV NS2B-NS3

This notebook processes the ZIKV NS2B-NS3 sample preparation data to create Echo transfer CSV files batched by plate-ID. Each output file contains the necessary columns for Echo liquid handling system transfers.

## Process Overview:
1. Load sample prep data from CSV
2. Group transfers by plate-ID
3. Generate destination wells (A1 through P24 for 384-well plates)
4. Create individual transfer files for each plate
5. Generate summary report

## 1. Import Required Libraries

In [1]:
import pandas as pd
import re
from pathlib import Path

print("Libraries imported successfully!")

Libraries imported successfully!


## 2. Configuration Setup

In [2]:
# Configuration settings
PLATENAME = "Zika-NS2B-NS3"
PLATETYPE = "creoptix"
INPUT_FILE = "2026-02(feb)-18-Zika-NS2B ASAP sample prep file.csv"  # CSV file with columns: 'Well', 'Compound', 'Concentration'
OUTPUT_DIR = "transfer-files"  # Current directory
GENERATE_CREOPTIX_FILES = True  # Set to True to generate Creoptix platemap files

# File paths
current_dir = Path(__file__).parent if '__file__' in globals() else Path('.')
input_path = current_dir / INPUT_FILE
output_dir = Path(OUTPUT_DIR)

print(f"Configuration loaded:")
print(f"  Plate Name: {PLATENAME}")
print(f"  Plate Type: {PLATETYPE}")
print(f"  Input File: {input_path}")
print(f"  Output Directory: {output_dir}")
print(f"  Generate Creoptix files: {GENERATE_CREOPTIX_FILES}")

# Ensure output directory exists
output_dir.mkdir(exist_ok=True)

Configuration loaded:
  Plate Name: Zika-NS2B-NS3
  Plate Type: creoptix
  Input File: 2026-02(feb)-18-Zika-NS2B ASAP sample prep file.csv
  Output Directory: transfer-files
  Generate Creoptix files: True


## 3. Load Sample Prep Data

In [3]:
# Load the sample prep data
try:
    df = pd.read_csv(input_path)
    print(f"Successfully loaded {len(df)} rows of data")
    print(f"Columns: {list(df.columns)}")
    
    # Display basic info about the data
    print(f"\nUnique source plates found: {df['source-plate-ID'].nunique()}")
    print(f"Source plate IDs: {sorted(df['source-plate-ID'].unique())}")
    
    # Show first few rows
    print(f"\nFirst 5 rows:")
    print(df.head())
    
except FileNotFoundError:
    print(f"Error: Could not find input file at {input_path}")
    print("Please ensure the file exists in the current directory.")
except Exception as e:
    print(f"Error loading data: {e}")

Successfully loaded 3237 rows of data
Columns: ['source-plate-ID', 'source-well', 'sample-ID', 'transfer-volume(nL)']

Unique source plates found: 84
Source plate IDs: ['1530852-Y4-110', '1530852-Y4-113', '1530852-Y4-116', '1530852-Y4-118', '1530852-Y4-122', '1530852-Y4-123', '1530852-Y4-127', '1530852-Y4-128', '1530852-Y4-132', '1530852-Y4-135', '1530852-Y4-137', '1530852-Y4-138', '1530852-Y4-142', '1530852-Y4-143', '1530852-Y4-147', '1530852-Y4-148', '1530852-Y4-152', '1530852-Y4-154', '1530852-Y4-158', '1530852-Y4-162', '1530852-Y4-166', '1530852-Y4-167', '1530852-Y4-170', '1530852-Y4-174', '1530852-Y4-179', '1530852-Y4-183', '1530852-Y4-187', '1530852-Y4-192', '1530852-Y4-195', '1530852-Y4-199', '1530852-Y4-200', '1530852-Y4-205', '1530852-Y4-206', '1530852-Y4-210', '1530852-Y4-213', '1530852-Y4-214', '1530852-Y4-217', '1530852-Y4-218', '1530852-Y4-221', '1530852-Y4-224', '1530852-Y4-228', '1530852-Y4-231', '1530852-Y4-235', '1530852-Y4-238', '1530852-Y4-242', '1530852-Y4-243', '15

## 4. Define Helper Functions

In [4]:
def generate_plate_batch_name(plate_number):
    """Generate plate batch name using the format: platename + platetype + _p## """
    return f"{PLATENAME}_{PLATETYPE}_p{plate_number:02d}"

def normalize_well_format(well):
    """Convert well format to standard format (e.g., A05 -> A5, AC05 -> AC5)"""
    if pd.isna(well):
        return well
    
    # Extract letter part and number part
    well_str = str(well).strip()
    
    # Handle multi-letter rows (like AA, AB, AC, etc.)
    match = re.match(r'([A-Z]+)(\d+)', well_str.upper())
    if match:
        letters, numbers = match.groups()
        # Remove leading zeros from numbers
        return f"{letters}{int(numbers)}"
    
    return well_str

def generate_destination_wells(num_wells):
    """Generate destination wells in order from A1 through P24 for 384-well plates"""
    wells = []
    rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']
    cols = list(range(1, 25))  # 1 to 24
    
    # Generate wells in column-first order (A1, B1, C1, ... P1, A2, B2, ...)
    for col in cols:
        for row in rows:
            wells.append(f"{row}{col}")
            if len(wells) >= num_wells:
                return wells[:num_wells]
    
    # If we need more than 384 wells, this would be an error
    if num_wells > len(wells):
        print(f"Warning: Requested {num_wells} wells but 384-well plate only has {len(wells)} wells")
    
    return wells[:num_wells]

# Test the functions
print("Testing helper functions:")
print(f"Plate batch name example: {generate_plate_batch_name(1)}")
print(f"Well format examples: {normalize_well_format('A05')} -> {normalize_well_format('AC05')}")
print(f"First 10 destination wells: {generate_destination_wells(10)}")

Testing helper functions:
Plate batch name example: Zika-NS2B-NS3_creoptix_p01
Well format examples: A5 -> AC5
First 10 destination wells: ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1', 'J1']


## 5. Process Each Plate and Create Transfer Files

In [5]:
# Main processing loop with continuous destination plate filling
summary_data = []
current_destination_plate = 1
current_destination_well_index = 0
max_wells_per_plate = 384

# Pre-generate all possible destination wells for 384-well plates
rows = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']
cols = list(range(1, 25))  # 1 to 24
all_destination_wells = []
for col in cols:
    for row in rows:
        all_destination_wells.append(f"{row}{col}")

# Group by source-plate-ID and process each plate in the order they appear in the sample prep file
unique_plates = df['source-plate-ID'].unique()
print(f"Processing {len(unique_plates)} unique source plates in file order...")
print(f"Total transfers to process: {len(df)}")
print(f"Estimated destination plates needed: {(len(df) + max_wells_per_plate - 1) // max_wells_per_plate}")
print("=" * 60)

for input_plate_num, plate_id in enumerate(unique_plates, 1):
    # Filter data for this source plate
    plate_data = df[df['source-plate-ID'] == plate_id].copy()
    num_transfers = len(plate_data)
    
    # Build transfer records for this source plate
    transfer_records = []
    
    for _, row in plate_data.iterrows():
        # Check if we need to move to a new destination plate
        if current_destination_well_index >= max_wells_per_plate:
            current_destination_plate += 1
            current_destination_well_index = 0
        
        # Get current destination plate batch name
        plate_batch_name = generate_plate_batch_name(current_destination_plate)
        
        # Get current destination well
        destination_well = all_destination_wells[current_destination_well_index]
        
        # Create transfer record
        transfer_records.append({
            'PlateBatch': plate_batch_name,
            'SrcWell': normalize_well_format(row['source-well']),
            'Destination well': destination_well,
            'XferVol': row['transfer-volume(nL)']
        })
        
        # Move to next destination well
        current_destination_well_index += 1
    
    # Create transfer dataframe for this source plate
    transfer_df = pd.DataFrame(transfer_records)
    
    # Create output filename
    output_filename = f"plate-{plate_id}-transfers.csv"
    output_path = output_dir / output_filename
    
    # Save to CSV
    transfer_df.to_csv(output_path, index=False)
    
    # Collect statistics for summary
    plate_batches_used = transfer_df['PlateBatch'].unique()
    first_well = transfer_records[0]['Destination well']
    last_well = transfer_records[-1]['Destination well']
    
    # Store summary information
    summary_data.append({
        'plate_id': plate_id,
        'plate_batch_names': list(plate_batches_used),
        'num_transfers': num_transfers,
        'output_file': output_filename,
        'destination_wells_range': f"{first_well} to {last_well}"
    })
    
    # Print progress
    print(f"✓ Source Plate {input_plate_num:2d}: {plate_id}")
    print(f"  - Destination plate(s): {', '.join(plate_batches_used)}")
    print(f"  - Transfers: {num_transfers}")
    print(f"  - Destination wells: {first_well} to {last_well}")
    print(f"  - Output file: {output_filename}")
    print()

print(f"Processing complete! Created {len(summary_data)} transfer files.")
print(f"Total destination plates used: {current_destination_plate}")
print(f"Final destination well position: {current_destination_well_index}/{max_wells_per_plate}")
print(f"Last destination plate utilization: {(current_destination_well_index / max_wells_per_plate * 100):.1f}%")

Processing 84 unique source plates in file order...
Total transfers to process: 3237
Estimated destination plates needed: 9
✓ Source Plate  1: 1530852-Y4-59
  - Destination plate(s): Zika-NS2B-NS3_creoptix_p01
  - Transfers: 1
  - Destination wells: A1 to A1
  - Output file: plate-1530852-Y4-59-transfers.csv

✓ Source Plate  2: 1530852-Y4-95
  - Destination plate(s): Zika-NS2B-NS3_creoptix_p01
  - Transfers: 2
  - Destination wells: B1 to C1
  - Output file: plate-1530852-Y4-95-transfers.csv

✓ Source Plate  3: 1530852-Y4-118
  - Destination plate(s): Zika-NS2B-NS3_creoptix_p01
  - Transfers: 2
  - Destination wells: D1 to E1
  - Output file: plate-1530852-Y4-118-transfers.csv

✓ Source Plate  4: 1530852-Y4-122
  - Destination plate(s): Zika-NS2B-NS3_creoptix_p01
  - Transfers: 8
  - Destination wells: F1 to M1
  - Output file: plate-1530852-Y4-122-transfers.csv

✓ Source Plate  5: 1530852-Y4-128
  - Destination plate(s): Zika-NS2B-NS3_creoptix_p01
  - Transfers: 2
  - Destination well

## 6. Generate Creoptix Platemap Files (Optional)

In [6]:
if GENERATE_CREOPTIX_FILES:
    print("Generating Creoptix platemap files...")
    print("=" * 60)
    
    # Create a mapping of destination wells to sample IDs
    # We need to go through all transfers and build this mapping using the SAME order as main processing
    destination_plate_maps = {}  # Key: plate_batch_name, Value: dict of well -> sample_id
    
    # Re-process the data to build the platemap - MUST follow same order as main processing
    current_destination_plate = 1
    current_destination_well_index = 0
    
    # Initialize the first destination plate
    current_plate_name = generate_plate_batch_name(current_destination_plate)
    destination_plate_maps[current_plate_name] = {}
    
    # Process plates in the SAME order as main transfer processing (grouped by source-plate-ID in file order)
    unique_plates = df['source-plate-ID'].unique()
    
    for input_plate_num, plate_id in enumerate(unique_plates, 1):
        # Filter data for this source plate - SAME as main processing
        plate_data = df[df['source-plate-ID'] == plate_id].copy()
        
        # Process each row in this source plate
        for _, row in plate_data.iterrows():
            # Check if we need to move to a new destination plate
            if current_destination_well_index >= max_wells_per_plate:
                current_destination_plate += 1
                current_destination_well_index = 0
                current_plate_name = generate_plate_batch_name(current_destination_plate)
                destination_plate_maps[current_plate_name] = {}
            
            # Get current destination well and sample ID
            destination_well = all_destination_wells[current_destination_well_index]
            sample_id = row['sample-ID']
            
            # Add to platemap
            destination_plate_maps[current_plate_name][destination_well] = sample_id
            
            # Move to next destination well
            current_destination_well_index += 1
    
    # Generate a Creoptix platemap file for each destination plate
    for plate_name, well_mapping in destination_plate_maps.items():
        # Create full 384-well plate template
        creoptix_data = []
        
        for col in range(1, 25):  # Columns 1-24
            for row in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P']:
                well = f"{row}{col}"
                designation = well_mapping.get(well, "")  # Empty if no sample in this well
                
                creoptix_data.append({
                    'Pos': well,
                    'Designation': designation,
                    'Concentration': '',
                    'MW': '',
                    'Volume': ''
                })
        
        # Create DataFrame and save
        creoptix_df = pd.DataFrame(creoptix_data)
        creoptix_filename = f"{plate_name}_creoptix_platemap.csv"
        creoptix_path = output_dir / creoptix_filename
        
        creoptix_df.to_csv(creoptix_path, index=False)
        
        # Count how many wells are populated
        populated_wells = sum(1 for v in well_mapping.values() if v)
        
        print(f"✓ Created Creoptix platemap: {creoptix_filename}")
        print(f"  - Populated wells: {populated_wells}/384")
        print()
    
    print(f"Generated {len(destination_plate_maps)} Creoptix platemap files.")
    print("=" * 60)
else:
    print("Skipping Creoptix platemap generation (GENERATE_CREOPTIX_FILES = False)")
    print("=" * 60)

Generating Creoptix platemap files...
✓ Created Creoptix platemap: Zika-NS2B-NS3_creoptix_p01_creoptix_platemap.csv
  - Populated wells: 384/384

✓ Created Creoptix platemap: Zika-NS2B-NS3_creoptix_p02_creoptix_platemap.csv
  - Populated wells: 384/384

✓ Created Creoptix platemap: Zika-NS2B-NS3_creoptix_p03_creoptix_platemap.csv
  - Populated wells: 384/384

✓ Created Creoptix platemap: Zika-NS2B-NS3_creoptix_p04_creoptix_platemap.csv
  - Populated wells: 384/384

✓ Created Creoptix platemap: Zika-NS2B-NS3_creoptix_p05_creoptix_platemap.csv
  - Populated wells: 384/384

✓ Created Creoptix platemap: Zika-NS2B-NS3_creoptix_p06_creoptix_platemap.csv
  - Populated wells: 384/384

✓ Created Creoptix platemap: Zika-NS2B-NS3_creoptix_p07_creoptix_platemap.csv
  - Populated wells: 384/384

✓ Created Creoptix platemap: Zika-NS2B-NS3_creoptix_p08_creoptix_platemap.csv
  - Populated wells: 384/384

✓ Created Creoptix platemap: Zika-NS2B-NS3_creoptix_p09_creoptix_platemap.csv
  - Populated wells:

## 7. Write Summary Report to Text File

In [7]:
# Write the final summary report to a text file
from datetime import datetime

# Create summary DataFrame from the summary data
summary_df = pd.DataFrame(summary_data)

# Create the report content
report_lines = []
report_lines.append("=" * 100)
report_lines.append("FINAL SUMMARY REPORT")
report_lines.append("Echo Transfer File Processing")
report_lines.append(f"Generated on: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
report_lines.append("=" * 100)

report_lines.append(f"\nTotal source plates processed: {len(summary_df)}")
report_lines.append(f"Total transfers created: {summary_df['num_transfers'].sum()}")
report_lines.append(f"Total destination plates used: {current_destination_plate}")

# Calculate final destination plate utilization
final_utilization = (current_destination_well_index / max_wells_per_plate) * 100 if current_destination_well_index > 0 else 0
report_lines.append(f"Final destination plate utilization: {current_destination_well_index}/{max_wells_per_plate} wells ({final_utilization:.1f}%)")

report_lines.append(f"\nDetailed breakdown by source plate:")
report_lines.append("-" * 100)
report_lines.append(f"{'Source Plate ID':<20} {'Destination Plate(s)':<35} {'Transfers':<10} {'Well Range':<20} {'Output File'}")
report_lines.append("-" * 100)

for _, row in summary_df.iterrows():
    dest_plates_str = ', '.join(row['plate_batch_names'])
    if len(dest_plates_str) > 33:
        dest_plates_str = dest_plates_str[:30] + "..."
    
    report_lines.append(f"{row['plate_id']:<20} {dest_plates_str:<35} {row['num_transfers']:<10} {row['destination_wells_range']:<20} {row['output_file']}")

report_lines.append("-" * 100)

# Statistics
report_lines.append(f"\nTransfer Statistics:")
report_lines.append(f"  - Minimum transfers per source plate: {summary_df['num_transfers'].min()}")
report_lines.append(f"  - Maximum transfers per source plate: {summary_df['num_transfers'].max()}")
report_lines.append(f"  - Average transfers per source plate: {summary_df['num_transfers'].mean():.1f}")

# Destination plate summary
all_dest_plates = set()
for plate_batch_names in summary_df['plate_batch_names']:
    all_dest_plates.update(plate_batch_names)

report_lines.append(f"\nDestination Plate Summary:")
report_lines.append(f"  Total destination plates created: {len(all_dest_plates)}")
for plate_name in sorted(all_dest_plates):
    # Count how many source plates contributed to this destination plate
    source_plates_using_dest = []
    for _, row in summary_df.iterrows():
        if plate_name in row['plate_batch_names']:
            source_plates_using_dest.append(row['plate_id'])
    
    report_lines.append(f"  - {plate_name}: Receives transfers from {len(source_plates_using_dest)} source plate(s)")

report_lines.append(f"\nAll transfer files have been saved in: {output_dir.absolute()}")

# Write to text file
report_filename = "Echo_Transfer_Summary_Report.txt"
report_path = output_dir / report_filename

with open(report_path, 'w') as f:
    f.write('\n'.join(report_lines))

print(f"✓ Summary report written to: {report_path}")
print(f"✓ Report contains {len(report_lines)} lines")

# Also display the content
print(f"\nReport content preview:")
print('\n'.join(report_lines))

✓ Summary report written to: transfer-files/Echo_Transfer_Summary_Report.txt
✓ Report contains 114 lines

Report content preview:
FINAL SUMMARY REPORT
Echo Transfer File Processing
Generated on: 2026-02-18 08:29:57

Total source plates processed: 84
Total transfers created: 3237
Total destination plates used: 9
Final destination plate utilization: 165/384 wells (43.0%)

Detailed breakdown by source plate:
----------------------------------------------------------------------------------------------------
Source Plate ID      Destination Plate(s)                Transfers  Well Range           Output File
----------------------------------------------------------------------------------------------------
1530852-Y4-59        Zika-NS2B-NS3_creoptix_p01          1          A1 to A1             plate-1530852-Y4-59-transfers.csv
1530852-Y4-95        Zika-NS2B-NS3_creoptix_p01          2          B1 to C1             plate-1530852-Y4-95-transfers.csv
1530852-Y4-118       Zika-NS2B-NS3_creopti